In [1]:
import numpy as np
import xarray as xr

In [2]:
# open forcing file 
ds = xr.open_dataset('/glade/scratch/molina/LBC_CMIP6_1pctCO2_y1-165_GlobAnnAvg_0p5degLat_c180929.nc', 
                     decode_cf=False, mask_and_scale=False, decode_times=False, decode_timedelta=None,
                     use_cftime=None, concat_characters=None, decode_coords=None,)

In [3]:
# subset arrays to only contain the first 151 years
ds = ds.isel(time=slice(0,151 + 1))

In [4]:
ds

<xarray.Dataset>
Dimensions:      (lat: 360, lon: 288, time: 152, bounds: 2)
Coordinates:
  * lat          (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon          (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time         (time) float32 364.5 547.5 912.5 ... 5.493e+04 5.53e+04
Dimensions without coordinates: bounds
Data variables:
    time_bnds    (time, bounds) float32 ...
    date         (time) int32 ...
    CO2_LBC      (time, lat) float32 ...
    N2O_LBC      (time, lat) float32 ...
    CH4_LBC      (time, lat) float32 ...
    CFC11eq_LBC  (time, lat) float32 ...
    CF2CL2_LBC   (time, lat) float32 ...
Attributes:
    data_title:     \nLower boundary conditions for CMIP6 1pctCO2 simulations...
    data_summary:   \nTime-dependent mole fraction surface boundary condition...
    data_creator:   \nMike Mills, mmills@ucar.edu
    cesm_contact:   \nMike Mills, mmills@ucar.edu
    creation_date:  \nSat Sep 29 18:46:46 MDT 2018
    data_script:    \ncreated with program CMIP6_1pctCO2GlobAnnAvg.ncl\nfrom ...

In [5]:
# grab relevant arrays to use for updated co2 forcing

# co2 coordinates
lat = ds['lat'].values
time = ds['time'].values

# reverse order of co2
CO2_LBC = ds['CO2_LBC'].values[::-1]

In [6]:
# new co2 dataarray
new_co2 = xr.DataArray(CO2_LBC, coords={'time': time, 'lat': lat,}, dims=['time', 'lat'])

# add co2 attributes
new_co2.attrs['units'] = ds['CO2_LBC'].attrs['units']
new_co2.attrs['long_name'] = ds['CO2_LBC'].attrs['long_name']
new_co2.attrs['_FillValue'] = ds['CO2_LBC'].attrs['_FillValue']

# time attrs
new_co2.coords['lat'].attrs['long_name'] = ds['lat'].attrs['long_name']
new_co2.coords['lat'].attrs['units'] = ds['lat'].attrs['units']

# lat attrs
new_co2.coords['time'].attrs['long_name'] = ds['time'].attrs['long_name']
new_co2.coords['time'].attrs['units'] = ds['time'].attrs['units']
new_co2.coords['time'].attrs['calendar'] = ds['time'].attrs['calendar']
new_co2.coords['time'].attrs['bounds'] = ds['time'].attrs['bounds']
new_co2.coords['time'].attrs['_FillValue'] = ds['time'].attrs['_FillValue']

In [7]:
# update the co2 variable in the original dataset
ds['CO2_LBC'] = new_co2

In [8]:
# add update attributes to the main dataset
ds.attrs['updated_by'] = 'Maria J. Molina (molina@ucar.edu)'
ds.attrs['update_scope'] = 'reversed order of CO2_LBC using python xarray'
ds.attrs['updated_date'] = 'Wed Dec 08 MDT 2021'

In [9]:
ds # sanity check

<xarray.Dataset>
Dimensions:      (lat: 360, lon: 288, time: 152, bounds: 2)
Coordinates:
  * lat          (lat) float32 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon          (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time         (time) float32 364.5 547.5 912.5 ... 5.493e+04 5.53e+04
Dimensions without coordinates: bounds
Data variables:
    time_bnds    (time, bounds) float32 364.0 365.0 ... 5.512e+04 5.548e+04
    date         (time) int32 1231 10702 20702 30702 ... 1490702 1500702 1510702
    CO2_LBC      (time, lat) float32 0.001265 0.001265 ... 0.0002843 0.0002843
    N2O_LBC      (time, lat) float32 2.73e-07 2.73e-07 ... 2.73e-07 2.73e-07
    CH4_LBC      (time, lat) float32 8.082e-07 8.082e-07 ... 8.082e-07 8.082e-07
    CFC11eq_LBC  (time, lat) float32 3.211e-11 3.211e-11 ... 3.211e-11 3.211e-11
    CF2CL2_LBC   (time, lat) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
Attributes:
    data_title:     \nLower boundary conditions for CMIP6 1pctCO2 simulations...
    data_summary:   \nTime-dependent mole fraction surface boundary condition...
    data_creator:   \nMike Mills, mmills@ucar.edu
    cesm_contact:   \nMike Mills, mmills@ucar.edu
    creation_date:  \nSat Sep 29 18:46:46 MDT 2018
    data_script:    \ncreated with program CMIP6_1pctCO2GlobAnnAvg.ncl\nfrom ...
    updated_by:     Maria J. Molina (molina@ucar.edu)
    update_scope:   reversed order of CO2_LBC using python xarray
    updated_date:   Wed Dec 08 MDT 2021

In [10]:
# save the updated file
ds.to_netcdf('/glade/scratch/molina/LBC_CMIP6_1pctCO2_y1-165_GlobAnnAvg_0p5degLat_c180929_REV.nc', format='NETCDF3_64BIT')
#ds.to_netcdf('/glade/scratch/molina/LBC_CMIP6_1pctCO2_y1-165_GlobAnnAvg_0p5degLat_c180929_REV.nc', format='NETCDF3_CLASSIC') # other option

In [11]:
# checking file below

In [12]:
import netCDF4
netCDF4.Dataset('/glade/scratch/molina/LBC_CMIP6_1pctCO2_y1-165_GlobAnnAvg_0p5degLat_c180929_REV.nc')

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_64BIT_OFFSET data model, file format NETCDF3):
    data_title: 
Lower boundary conditions for CMIP6 1pctCO2 simulations with CAM atmosphere, years 1-165
    data_summary: 
Time-dependent mole fraction surface boundary conditions for CMIP6 1pctCO2 simulations.
LBCs for CO2, N2O, CH4, CFC11eq, and CFC12.
All species derived from year 1850 of file LBC_CMIP6_1pctCO2_y1-165_0p5degLat_c180929.nc
CO2 increased by 1% each year over previous year, starting year 2.
All other species repeat 1850 every year for years 1-165.
Year 0 copied from year 1 for all species..
Year 166 copied from year 165 for all species.
    data_creator: 
Mike Mills, mmills@ucar.edu
    cesm_contact: 
Mike Mills, mmills@ucar.edu
    creation_date: 
Sat Sep 29 18:46:46 MDT 2018
    data_script: 
created with program CMIP6_1pctCO2GlobAnnAvg.ncl
from original file LBC_CMIP6_1pctCO2_y1-165_0p5degLat_c180929.nc
https://svn.code.sf.net/p/codescripts/code/trunk/ncl/lbc/
   